In [41]:
#%%file poisson_2d.py
from manapy.ddm import readmesh
from manapy.ddm import Domain

from manapy.ast import Variable, LinearSystem
import numpy as np

from  numpy.linalg import solve

from scipy import sparse
from scipy.sparse.linalg import spsolve

import os

from triangle import dist, orthocenter

from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D

from mpi4py import MPI
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

# ... get the mesh directory
'''try:
    MESH_DIR = os.environ['MESH_DIR']
 
except:
    BASE_DIR = os.path.dirname(os.path.realpath(__file__))
    BASE_DIR = os.path.join(BASE_DIR , '..')
    MESH_DIR = os.path.join(BASE_DIR, 'mesh')
'''
filename = "../mesh/untitled.msh"
#filename = os.path.join(MESH_DIR, filename)

        
dim = 2
readmesh(filename, dim=dim, periodic=[0,0,0])
    
#Create the informations about cells, faces and nodes
domain = Domain(dim=dim)
    
cells   = domain.cells
nbcells = domain.nbcells
faces   = domain.faces
nodes   = domain.nodes
nbfaces = len(faces.mesure)

#print(faces.name)
"""

def solver(cells, nbcells, faces, nodes):
    
    row    = []
    col    = []
    values = []
    b = np.zeros(nbcells, dtype = np.float32 ) 
    for i in range(nbcells):
        # Compute orthocenter of current cell i
        x_k = orthocenter([ nodes.vertex[ cells.nodeid[i][k] ] for k in range(3) ])
        
        s = 0            
        for e in cells.faceid[i]:
            if faces.name[e] == 0:
                for j in cells.cellfid[i]:
                    if e in cells.faceid[j]:
                        x_L = orthocenter([ nodes.vertex[ cells.nodeid[j][k] ] for k in range(3) ])
                        p = faces.mesure[e] / dist(x_k, x_L)
                        row.append(i)
                        col.append(j)
                        values.append(p)
                        s -= p
            if faces.name[e] == 1:
                p = faces.mesure[e] / dist(x_k, faces.center[e])
                s -= p
                b[i] = - (20 * p)
            if faces.name[e] in  [2, 3, 4]:
                p = faces.mesure[e] / dist(x_k, faces.center[e])
                s -= p
        row.append(i)
        col.append(i)
        values.append(s)    
        
    mat = sparse.csr_matrix((values, (row, col)))
    u = spsolve(mat, b)
    
    return u
"""                
def solver(cells, nbcells, faces, nodes):
    
    center = np.zeros( (nbcells, 2), dtype = np.float32 )
    b      = np.zeros(      nbcells, dtype = np.float32 ) 
    A      = np.zeros( (nbcells, nbcells), dtype = np.float32 )
    
    #row    = np.zeros(    4*nbcells, dtype = np.int32   )
    #col    = np.zeros(    4*nbcells, dtype = np.int32   )
    #values = np.zeros(    4*nbcells, dtype = np.int32   )

    
    # Compute orthocenter for all cells
    for i in range(nbcells):
        center[i, :] = orthocenter([ nodes.vertex[ cells.nodeid[i][k] ] for k in range(3) ])
    
    for i in range(nbfaces):
        l_cell = faces.cellid[i][0]
        r_cell = faces.cellid[i][1]
        
        # Innerfaces
        if faces.name[i] == 0:
            p = faces.mesure[i] / dist(center[l_cell], center[r_cell])
            A[l_cell, l_cell] -= p
            A[r_cell, r_cell] -= p
            A[l_cell, r_cell]  = p
            A[r_cell, l_cell]  = p
        
        # in faces    
        if faces.name[i] == 1:
            for j in [l_cell, r_cell]:
                if j != -1:
                    p = faces.mesure[i] / dist(center[j], faces.center[i])
                    A[j, j] -= p
                    b[j]     = - (20 * p)
        
        # out, upper, bottom faces
        if faces.name[i] in [2, 3, 4]:
            for j in [l_cell, r_cell]:
                if j != -1:
                    p = faces.mesure[i] / dist(center[j], faces.center[i])
                    A[j, j] -= p
            
            
            
            
        
    #mat = sparse.csr_matrix((values, (row, col)))
    #u = spsolve(mat, b)
    u = solve(A, b)
    return u
                
u = solver(cells, nbcells, faces, nodes)
"""
def plot2D(x, y, u):
    fig = pyplot.figure(figsize=(11, 7), dpi=100)
    ax = fig.gca(projection='3d')
    X, Y = np.meshgrid(x, y)
    surf = ax.plot_surface(X, Y, u, rstride=1, cstride=1, cmap=cm.viridis,
            linewidth=0, antialiased=False)
    ax.view_init(30, 225)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$') 
"""    

#plot2D(x, y, u)

        
        
    
    
    
    
    
    
    
    
    
"""
f = lambda x, y, z : 25000. * (1. - x)
    fexact = np.zeros(nbcells)
    
    for i in range(nbcells):
        fexact[i] = f(cells.center[i][0], cells.center[i][1], 0.)
    errorl2 = w.norml2(fexact, order=1)  
    
    #compute error
    print("l2 norm is ", errorl2)    
"""    
    

Starting ....
Number of Cells :  198
Number of Nodes :  118


'\nf = lambda x, y, z : 25000. * (1. - x)\n    fexact = np.zeros(nbcells)\n    \n    for i in range(nbcells):\n        fexact[i] = f(cells.center[i][0], cells.center[i][1], 0.)\n    errorl2 = w.norml2(fexact, order=1)  \n    \n    #compute error\n    print("l2 norm is ", errorl2)    \n'

In [42]:
domain.save_on_cell(value = u)

 **************************** Computing ****************************
$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Saving Results $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Iteration =  0 time =  0 time step =  0
max w = 5.436275615e-315
